This is where we do the ISCE interferogram generation.

Reference: https://github.com/aria-jpl/ariamh/blob/alos2/interferogram/alos/create_alos2_ifg.py#L182-L219

In [1]:
from pathlib import Path
import rasterio
import numpy as np
import json
from jinja2 import Template
import site
from subprocess import check_call, check_output
import os
import sys

In [2]:
## Will be taken care of in papermill script
## If not, we must run jupyter from the `notebooks/` directory
if not os.environ.get('PGE_DIRECTORY'):
    path = Path(os.getcwd()).parents[0]
    os.environ['PGE_DIRECTORY'] = str(path.absolute())
    assert(path.name == 'alos2app_pge')

sys.path.append(os.environ['PGE_DIRECTORY'])
from ifg_tools.rio_tools import reproject_arr_to_match_profile
PGE_DIRECTORY = os.environ['PGE_DIRECTORY']

In [4]:
product_dir = list(Path('.').glob('ALOS2-GUNW-*'))[0]
prod_id = product_dir.name
prod_id

'ALOS2-GUNW-D-153-20150412-20150301-182629-18N-100W-v1_0_0'

In [5]:
metadata_path = product_dir/f'{prod_id}.met.json'
metadata = json.load(open(metadata_path, 'r'))
metadata

{'reference_metadata': {'slc_id': 'ALOS2047853215-150412',
  'production_level': 'WBDR1',
  'bbox_xy_coords': [[-99.54163571839928, 21.841270600310185],
   [-96.17730760260231, 21.25939696689257],
   [-96.843173553836, 18.068982542026124],
   [-100.13694111929185, 18.65929018004776]],
  'start_time': '2015-04-12T18:26:03.683087',
  'stop_time': '2015-04-12T18:26:55.705525',
  'absolute_orbit': '04785',
  'frame': '3215',
  'flight_direction': 'D',
  'satellite_name': 'ALOS2',
  'source': 'isce_preprocessing',
  'track': 153},
 'secondary_metadata': {'slc_id': 'ALOS2041643215-150301',
  'production_level': 'WBDR1',
  'bbox_xy_coords': [[-99.54225234548686, 21.84242172762871],
   [-96.17777138358394, 21.260454461136113],
   [-96.84383040237188, 18.07008971011115],
   [-100.13767953028737, 18.660475516270562]],
  'start_time': '2015-03-01T18:26:06.130416',
  'stop_time': '2015-03-01T18:26:58.152714',
  'absolute_orbit': '04164',
  'frame': '3215',
  'flight_direction': 'D',
  'satellite_n

In [6]:
ref_date_str = metadata['reference_date'].replace('-', '')[2:]
ref_date_str

'150412'

In [7]:
reference_dir = list(Path('.').glob(f'ALOS2*{ref_date_str}'))[0]
reference_dir

PosixPath('ALOS2047853200-150412')

In [8]:
sec_date_str = metadata['secondary_date'].replace('-', '')[2:]
sec_date_str

'150301'

In [9]:
secondary_dir = list(Path('.').glob(f'ALOS2*{sec_date_str}'))[0]
secondary_dir

PosixPath('ALOS2041643200-150301')

In [10]:
with open(f'{PGE_DIRECTORY}/alos2App_processing_template.xml', 'r') as file:
    template = Template(file.read())


In [11]:
alos2_xml = template.render(reference_dir=reference_dir,
                            secondary_dir=secondary_dir,
                            reference_frames=str([int(metadata['reference_metadata']['frame'])]),
                            secondary_frames=str([int(metadata['secondary_metadata']['frame'])]),
                            dem_for_geocoding='geocode_dem.wgs84',
                            dem_for_coregistration='preprocess_dem.wgs84',
                            reference_polarization=metadata['polarization'],
                            secondary_polarization=metadata['polarization'],
                            starting_swath=1,
                            ending_swath=5,
                           )
print(alos2_xml)

<?xml version="1.0" encoding="UTF-8"?>
<alos2App>
  <component name="alos2insar">
    <property name="reference directory">ALOS2047853200-150412</property>
    <property name="secondary directory">ALOS2041643200-150301</property>
    <property name="reference frames">[3215]</property>
    <property name="secondary frames">[3215]</property>
    <property name="starting swath">1</property>
    <property name="ending swath">5</property>
    <property name="reference polarization">HH</property>
    <property name="secondary polarization">HH</property>
    <property name="dem for coregistration">preprocess_dem.wgs84</property>
    <property name="dem for geocoding">geocode_dem.wgs84</property>
    <property name="geocode file list">['filt_*-*_30rlks_168alks.unw', 'filt_*-*_30rlks_168alks.unw.conncomp', 'filt_*-*_30rlks_168alks.int', '*-*_30rlks_168alks.phsig', '*-*_30rlks_168alks.cor', '*-*_30rlks_168alks.los', '*-*_30rlks_168alks.ion']</property>
    <property name="use GPU">False</propert

In [12]:
with open('alos2App.xml', "w") as file:
    file.write(alos2_xml)

In [13]:
isce_apps_path = site.getsitepackages()[0] + '/isce/applications'
alos_app_cmd = f'{isce_apps_path}/alos2App.py alos2App.xml'
alos_app_cmd

'/u/leffe-data2/cmarshak/miniconda3/envs/isce2/lib/python3.8/site-packages/isce/applications/alos2App.py alos2App.xml'

In [17]:
if isce_apps_path not in os.environ["PATH"]:
    os.environ["PATH"] += f':{isce_apps_path}'
os.environ["PATH"]

'/u/leffe-data2/cmarshak/miniconda3/envs/isce2/bin:/u/leffe-data2/cmarshak/miniconda3/bin:/u/leffe-data2/cmarshak/miniconda3/condabin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/home/cmarshak/bin:/usr/local/sbin:/usr/sbin:/home/cmarshak/bin:/u/leffe-data2/cmarshak/miniconda3/envs/isce2/lib/python3.8/site-packages/isce/applications'

Let's check that the path was correctly modified.

In [20]:
!imageMath.py

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
usage: imageMath.py [-h] [-H] -e EQUATION [-o OUT] [-s SCHEME] [-t DTYPE] [-d]
                    [-n]
imageMath.py: error: the following arguments are required: -e/--eval


In [ ]:
%%time

!alos2App.py alos2App.xml >> alos2app_processing.log 2> alos2app_error.log